# Using_an_SBML_model (python3)

This is a copy of Using_an_SBML_model (Python2) but built with a Python3 kernel to make sure everything works.

# Using an SBML model

## Getting started

### Installing libraries

Before you start, you will need to install a couple of libraries:
   
The [ModelSeedDatabase](https://github.com/ModelSEED/ModelSEEDDatabase) has all the biochemistry we'll need. You can install that with `git clone`.
   
The [PyFBA](http://linsalrob.github.io/PyFBA) library has detailed [installation instructions](http://linsalrob.github.io/PyFBA/installation.html). Don't be scared, its mostly just `pip install`.

(Optional) Also, get the [SEED Servers](https://github.com/linsalrob/SEED_Servers_Python) as you can get a lot of information from them. You can install the git python repo from github.  Make sure that the SEED_Servers_Python is in your PYTHONPATH.

We start with importing some modules that we are going to use. 

We import *sys* so that we can use standard out and standard error if we have some error messages.<br>
We import *copy* so that we can make a deep copy of data structures for later comparisons.<br>
Then we import the *PyFBA* module to get started.

In [1]:
import sys
import os
import copy
import PyFBA
import pickle

We are using /home/redwards/GitHubsLinux/PyFBA/venv/lib/python3.9/site-packages/PyFBA-2.4-py3.9.egg/PyFBA/Biochemistry/ModelSEEDDatabase for our data


## Running an SBML model

If you have run your genome through RAST, you can download the [SBML](http://www.sbml.org/) model and use that directly.

We have provided an [SBML model of *Citrobacter sedlakii*](https://raw.githubusercontent.com/linsalrob/PyFBA/master/example_data/Citrobacter/Citrobacter_sedlakii.sbml) that you can download and use. You can right-ctrl click on this link and save the SBML file in the same location you are running this iPython notebook.

We use this SBML model to demonstrate the key points of the FBA approach: defining the reactions, including the boundary, or drainflux, reactions; the compounds, including the drain compounds; the media; and the reaction bounds. 

We'll take it step by step!

We start by parsing the model:

In [2]:
sbml = PyFBA.parse.parse_sbml_file("../example_data/Citrobacter/Citrobacter_sedlakii.sbml")

We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/PyFBA.2021-06-07T15:02:41.432554.log
Added compound cpd00060 | L_Methionine
Added compound cpd00067 | H
Added compound cpd00001 | H2O
Added compound cpd00035 | L_Alanine
Added compound cpd11590 | met_L_ala_L
Added compound cpd00161 | L_Threonine
Added compound cpd11582 | ala_L_Thr_L
Added compound cpd11589 | gly_asp_L
Added compound cpd00041 | L_Aspartate
Added compound cpd00033 | Glycine
Added compound cpd00084 | L_Cysteine
Added compound cpd15603 | Gly_Cys
Added compound cpd00023 | L_Glutamate
Added compound cpd11586 | ala_L_glu_L
Added compound cpd00132 | L_Asparagine
Added compound cpd11581 | gly_asn_L
Added compound cpd15604 | Gly_Leu
Added compound cpd00107 | L_Leucine
Added compound cpd11585 | L_alanylglycine
Added compound cpd11593 | ala_L_asp_L
Added compound cpd01017 | Cys_Gly
Added compound cpd00053 | L_Glutamine
Added compound cpd11580 | Gly_Gln
Added compound cpd00066 | L_Phenylalanine
Added compound cpd1

### Find all the reactions and identify those that are boundary reactions

We need a set of reactions to run in the model. In this case, we are going to run all the reactions in our SBML file. However, you can change this set if you want to knock out reactions, add reactions, or generally modify the model. We store those in the `reactions_to_run` set.

The boundary reactions refer to compounds that are secreted but then need to be removed from the `reactions_to_run` set. We usually include a consumption of those compounds that is open ended, as if they are draining away. We store those reactions in the `uptake_secretion_reactions` dictionary.


In [3]:
# Get a dict of reactions.
# The key is the reaction ID, and the value is a metabolism.reaction.Reaction object
reactions = sbml.reactions
reactions_to_run = set()
uptake_secretion_reactions = {}
biomass_equation = None
for r in reactions:
    if 'biomass_equation' == r:
        biomass_equation = reactions[r]
        print(f"Our biomass equation is {biomass_equation.readable_name}")
        continue
    is_boundary = False
    for c in reactions[r].all_compounds():
        if c.uptake_secretion:
            is_boundary = True
            break
    if is_boundary:
        reactions[r].is_uptake_secretion = True
        uptake_secretion_reactions[r] = reactions[r]
    else:
        reactions_to_run.add(r)


Our biomass equation is Citrobacter_sedlakii_119_auto_biomass


At this point, we can take a look at how many reactions are in the model, not counting the biomass reaction:

In [4]:
print(f"The biomass equation is {biomass_equation}")
print("There are {} reactions in the model".format(len(reactions)))
print("There are {}".format(len(uptake_secretion_reactions)),
      "uptake/secretion reactions in the model")
print("There are {}".format(len(reactions_to_run)),
      "reactions to be run in the model")

The biomass equation is biomass_equation: Citrobacter_sedlakii_119_auto_biomass
There are 1574 reactions in the model
There are 0 uptake/secretion reactions in the model
There are 1573 reactions to be run in the model


In [5]:
import pickle
with open('rgood.txt', 'w') as out:
    for r in reactions:
        out.write(f"{r}\n")

### Find all the compounds in the model, and filter out those that are secreted

We need to filter out uptake and secretion compounds from our list of all compounds before we can make a stoichiometric matrix.

In [6]:
# Get a dict of compounds. 
# The key is the string representation of the compound and
# the value is a metabolite.compound.Compound object
# Get a dict of compounds. 
# The key is the string representation of the compound and
# the value is a metabolite.compound.Compound object
all_compounds = sbml.compounds
# Filter for compounds that are boundary compounds
filtered_compounds = set()
for c in all_compounds:
    if not c.uptake_secretion:
        filtered_compounds.add(c)

Again, we can see how many compounds there are in the model.

In [7]:
print("There are {} total compounds in the model".format(len(all_compounds)))
print("There are {}".format(len(filtered_compounds)),
      "compounds that are not involved in uptake and secretion")

There are 2613 total compounds in the model
There are 2439 compounds that are not involved in uptake and secretion


And now we have the size of our stoichiometric matrix! Notice that the stoichiometric matrix is composed of the reactions that we are going to run and the compounds that are in those reactions (but not the uptake/secretion reactions and compounds).

In [8]:
print(f"The stoichiometric matrix will be {len(reactions_to_run):,} reactions by {len(filtered_compounds):,} compounds")

The stoichiometric matrix will be 1,573 reactions by 2,439 compounds


### Read the media file, and correct the media names

In our [media](https://github.com/linsalrob/PyFBA/tree/master/media) directory, we have a lot of different media formulations, most of which we use with the Genotype-Phenotype project. For this example, we are going to use Lysogeny Broth (LB). There are many different formulations of LB, but we have included the recipe created by the folks at Argonne so that it is comparable with their analysis. You can download [ArgonneLB.txt](https://raw.githubusercontent.com/linsalrob/PyFBA/master/media/ArgonneLB.txt) and put it in the same directory as this iPython notebook to run it.

Once we have read the file we need to correct the names in the compounds. Sometimes when compound names are exported to the SBML file they are modified slightly. This just corrects those names.

In [9]:
# Read the media file
media = PyFBA.parse.read_media_file("/home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/Biochemistry/media/ArgonneLB.txt")
# Correct the names
media = sbml.correct_media(media)
print(f"The media has {len(media)} compounds")

The media has 1 compounds


In [10]:
print(f"There are {len(filtered_compounds):,} filtered compounds, {len(all_compounds):,} all compounds, and {len(reactions):,} reactions")

There are 2,439 filtered compounds, 2,613 all compounds, and 1,574 reactions


### Set the reaction bounds for uptake/secretion compounds

The uptake and secretion compounds typically have reaction bounds that allow them to be consumed (i.e. diffuse away from the cell) but not produced. However, our media components can also increase in concentration (i.e. diffuse to the cell) and thus the bounds are set higher. Whenever you change the growth media, you also need to adjust the reaction bounds to ensure that the media can be consumed!


In [11]:
# Adjust the lower bounds of uptake secretion reactions
# for things that are not in the media
for u in uptake_secretion_reactions:
    is_media_component = False
    for c in uptake_secretion_reactions[u].all_compounds():
        if c in media:
            is_media_component = True
    if not is_media_component:
        reactions[u].lower_bound = 0.0
        uptake_secretion_reactions[u].lower_bound = 0.0

### Run the FBA

Now that we have constructed our model, we can run the FBA!

In [12]:
status, value, growth = PyFBA.fba.run_fba(filtered_compounds, reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


Set {'id': 'Thymine_b', 'name': 'e', 'reactions': set(), 'model_seed_id': 'cpd00151_b', 'alternate_seed_ids': set(), 'abbreviation': 'cpd00151_b', 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': '1', 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': True, 'is_core': False, 'inchikey': 0, 'location': 'e'}
Read {'id': 'Thymine_b', 'name': 'e', 'reactions': set(), 'model_seed_id': 'cpd00151_b', 'alternate_seed_ids': set(), 'abbreviation': 'cpd00151_b', 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': '1', 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': True, 'is_core': False, 'inchikey': 0, 'location': 'e'}
In the model there are : 1139 compounds and 1575 reactions
We are loading 1139 rows and 1575 columns


The FBA completed with a flux value of 481.4230689836746 --> growth: True


Length of the media: 1
Number of reactions to run: 1573
Number of compounds in SM: 1139
Number of reactions in SM: 1575
Revised number of total reactions: 1575
Number of total compounds: 2440
SMat dimensions: 1139 x 1575


# Export the components of the model

I am trying to compare this model to one built just from reactions, and so to see what's working, I'm going to export all the components and then import them as need

In [13]:
pickle.dump(filtered_compounds, open('compounds.pickle', 'wb'))
pickle.dump(reactions, open('reactions.pickle', 'wb'))
pickle.dump(reactions_to_run, open('reactions_to_run.pickle', 'wb'))
pickle.dump(media, open('media.pickle', 'wb'))
pickle.dump(biomass_equation, open('sbml_biomass.pickle', 'wb'))
pickle.dump(uptake_secretion_reactions, open('uptake_secretion_reactions.pickle', 'wb'))

Set {'id': 'Nitrite', 'name': 'c', 'reactions': set(), 'model_seed_id': 'cpd00075', 'alternate_seed_ids': set(), 'abbreviation': 'cpd00075_c0', 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': '0', 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': False, 'is_core': False, 'inchikey': 0, 'location': 'cpd00075'}
Set {'id': 'cpd15339', 'name': 'c0', 'reactions': set(), 'model_seed_id': 'cpd15339', 'alternate_seed_ids': set(), 'abbreviation': None, 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': 0, 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': False, 'is_core': False, 'inchikey': 0, 'location': 'smbl2'}
Set {'id': 'cpd00672', 'name': 'c0', 'reactions': set(), 'model_seed_id': 'cpd00672', 'alternate_seed_ids': set(), 'abbreviation': None, 'aliases': None, 'formula': None,

Set {'id': 'cpd15540', 'name': 'c0', 'reactions': set(), 'model_seed_id': 'cpd15540', 'alternate_seed_ids': set(), 'abbreviation': None, 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': 0, 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': False, 'is_core': False, 'inchikey': 0, 'location': 'smbl2'}
Set {'id': 'cpd15721', 'name': 'c0', 'reactions': set(), 'model_seed_id': 'cpd15721', 'alternate_seed_ids': set(), 'abbreviation': None, 'aliases': None, 'formula': None, 'mw': 0, 'common': False, 'charge': 0, 'is_cofactor': False, 'linked_compound': False, 'pka': 0, 'pkb': 0, 'is_obsolete': False, 'abstract_compound': False, 'uptake_secretion': False, 'is_core': False, 'inchikey': 0, 'location': 'smbl1'}
Set {'id': 'cpd15727', 'name': 'c0', 'reactions': set(), 'model_seed_id': 'cpd15727', 'alternate_seed_ids': set(), 'abbreviation': None, 'aliases': None, 'formula': None, 'mw': 0, 'c

In [14]:
for c in filtered_compounds:
    if type(c) == 'PyFBA.metabolism.compound.CompoundWithLocation':
        print(f"{c.name}:: {c.id}")
    else:
        print(f"{type(c)}")

<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compou

<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compound.CompoundWithLocation'>
<class 'PyFBA.metabolism.compou

In [15]:
for c in filtered_compounds:
    if isinstance(c, PyFBA.metabolism.compound.CompoundWithLocation):
        if not c.id:
            print(f"No id for {c.name}")


In [16]:
for c in filtered_compounds:
    if isinstance(c, PyFBA.metabolism.compound.CompoundWithLocation):
        print(f"|{c.id}| {c.name}")

|Nitrite| c
|cpd15339| c0
|cpd00672| c0
|Undecaprenyl_diphospho_N_acetylmuramoyl_N_acetylglucosamine_L_alanyl_D_glutamyl_L_lysyl_D_alanyl_D_alanine| c
|cpd00478| c0
|Oxopent_4_enoate| c
|CDPglycerol| c
|Taurine| c
|cpd11493| c0
|1_Acyl_sn_glycero_3_phosphoglycerol_octadecanoyl| c
|2E_Hexadecenoyl_CoA| c
|gly_asp_L| e
|9_methyl_3_oxo_decanoyl_ACP| c
|L_Alanine| c
|phosphoribulosylformimino_AICAR_phosphate| c
|cpd00540| c0
|dTDP_4_acetamido_4_6_dideoxy_D_galactose| c
|Cbl| c
|cpd15522| c0
|cpd03917| c0
|cpd02160| c0
|cpd03697| c0
|cpd00421| c0
|phosphatidylserine_dihexadec_9_enoyl| c
|cpd15797| c0
|Farnesyldiphosphate| c
|cpd01133| c0
|2_Acyl_sn_glycero_3_phosphoethanolamine_octadec_11_enoyl| c
|cpd00101| c0
|L_Fuculose| c
|cpd11514| c0
|cpd01563| c0
|cpd00103| c0
|Glycolaldehyde| c
|UDP_N_acetylmuramoyl_L_alanine| c
|cpd01401| c0
|cpd00079| c0
|cpd01912| e0
|cpd15605| c0
|O2_| c
|Lactate| e
|Adenosyl_cobinamide_phosphate| c
|cpd00465| c0
|cpd08353| c0
|3_Deoxy_D_manno_octulosonate8_phos

|cpd00480| c0
|Vitamin_B12| e
|1_Acyl_sn_glycero_3_phosphoglycerol_tetradecanoyl| c
|cpd00531| c0
|Ornithine| e
|cpd00141| c0
|cpd01844| c0
|fa11coa| c
|12_methyl_tetra_decanoyl_ACP| c
|Uracil| e
|L_Threonine| c
|cpd15342| c0
|cpd00216| c0
|cpd02612| c0
|cpd16540| c0
|cpd15355| c0
|cpd00280| c0
|Deoxyguanosine| c
|cpd01092| c0
|cpd11520| c0
|Peptide_L_methionine| c
|Glycerol| c
|cpd00030| e0
|cpd11539| c0
|Triphosphate| c
|cpd00202| c0
|Fe2| e
|cpd11532| c0
|cpd00363| c0
|cpd02547| c0
|cpd02678| c0
|cpd11467| c0
|cpd01651| c0
|3_oxoadipate_enol_lactone| c
|Cobyrinate| c
|PAN| e
|cpd15417| c0
|cpd00143| c0
|Hexanoyl_ACP| c
|S_Adenosylmethioninamine| c
|Salicin_6P| c
|cpd11507| c0
|Myristoyllipoteichoic_acid_n24__linked__unsubstituted| c
|cpd02642| c0
|cpd15536| c0
|cpd15345| c0
|cpd00822| c0
|8_methyl_nonanoyl_ACP| c
|R_3_Hydroxybutanoate| c
|cpd00159| c0
|GTP| c
|cpd15528| c0
|cpd03443| c0
|cpd00150| c0
|cpd02394| c0
|9_methyl_trans_dec_2_enoyl_ACP| c
|cpd00396| e0
|D_Glucose| c
|croto

In [17]:
for c in sbml.compounds:
    if isinstance(c, PyFBA.metabolism.compound.CompoundWithLocation):
        print(f"|{c.id}| {c.name}")

|Nitrite| c
|cpd15339| c0
|cpd00672| c0
|Undecaprenyl_diphospho_N_acetylmuramoyl_N_acetylglucosamine_L_alanyl_D_glutamyl_L_lysyl_D_alanyl_D_alanine| c
|cpd00478| c0
|Oxopent_4_enoate| c
|CDPglycerol| c
|Taurine| c
|cpd11493| c0
|1_Acyl_sn_glycero_3_phosphoglycerol_octadecanoyl| c
|2E_Hexadecenoyl_CoA| c
|gly_asp_L| e
|9_methyl_3_oxo_decanoyl_ACP| c
|L_Alanine| c
|phosphoribulosylformimino_AICAR_phosphate| c
|cpd00540| c0
|dTDP_4_acetamido_4_6_dideoxy_D_galactose| c
|Cbl| c
|cpd15522| c0
|cpd03917| c0
|cpd02160| c0
|cpd03697| c0
|cpd00421| c0
|phosphatidylserine_dihexadec_9_enoyl| c
|cpd15797| c0
|Farnesyldiphosphate| c
|cpd01133| c0
|2_Acyl_sn_glycero_3_phosphoethanolamine_octadec_11_enoyl| c
|cpd00101| c0
|L_Fuculose| c
|L_Methionine_b| e
|cpd11514| c0
|cpd01563| c0
|cpd00103| c0
|Glycolaldehyde| c
|UDP_N_acetylmuramoyl_L_alanine| c
|cpd01401| c0
|cpd00079| c0
|cpd01912| e0
|O2_| c
|cpd15605| c0
|Lactate| e
|Adenosyl_cobinamide_phosphate| c
|cpd00465| c0
|cpd08353| c0
|3_Deoxy_D_manno

|5_methyl_3_hydroxy_hexanoyl_ACP| c
|cpd00334| c0
|2_Keto_3_deoxy_6_phosphogluconate| c
|2_Dehydro_3_deoxy_D_glucarate| c
|cpd00221| c0
|Calomide| c
|Dimethylbenzimidazole| c
|Phosphatidylglycerophosphate_dioctadecanoyl| c
|cpd00215| c0
|Ubiquinol_8| c
|Mg| e
|Methacrylyl_CoA| c
|maltose_6_phosphate| c
|cpd14905| c0
|1_dodecanoyl_sn_glycerol_3_phosphate| c
|cpd11474| c0
|3_Isopropylmalate| c
|cpd15353| c0
|cpd15477| c0
|L_Proline| e
|Dihydrodipicolinate| c
|Pyruvate_b| e
|2_Dehydro_D_gluconate| c
|2_3_Dihydroxy_isovalerate| c
|Deoxycytidine_b| e
|cpd15538| c0
|Octanoyl_CoA| c
|hexanesulfonate| c
|cpd00107| c0
|Hexadecenoyl_ACP| c
|Xylose| e
|2_Acyl_sn_glycero_3_phosphoglycerol_hexadec_9_enoyl| c
|5_Deoxy_glucuronic_acid| c
|2_methylbutyryl_ACP| c
|cpd11563| c0
|GSH| c
|D_Glucarate_b| e
|cpd00523| c0
|dTDP_rhamnose| c
|cpd00100| c0
|Glycerophosphoglycerol| c
|L_Arginine| c
|5_Phosphoribosyl_4_carboxy_5_aminoimidazole| c
|cpd00515| c0
|octadecenoate| c
|phosphatidylethanolamine_ditetrade